In [5]:
import pandas as pd
import sqlite3

pd.set_option('display.max_columns', None)

FEMA provides the NFIRS datasets as multiple CSV files covering multiple years. There are also two different table schemas for each year.

We'll load this data into a local SQLite db file, which will enable us to more easily geocode our addresses, allowing us to compare between the NFIRS and HUD REAC datasets.

For our purposes, we'll only load the "incidentaddress.txt" and "basicincident.txt" files into SQL. These files contain data on the location of the incidents and the nature of the incidents respectively.

Note: Before running this notebook, place the NFIRS fire incident data files in your working directory, and extract both the main folder and the subfolders.

In [9]:
IN_DIR = './data/nfirs/'
NFIRS_PATHS = [
    'nfirs_fire_hazmat_pdr_2021/nfirs_fire_hazmat_pdr_2021/NFIRS_FIRES_2021_111422',
    'nfirs_fire_hazmat_pdr_2020/nfirs_fire_hazmat_pdr_2020/NFIRS_FIRES_2020_022322',
    'usfa_nfirs_2019_hazmat/USFA NFIRS 2019 Hazmat/NFIRS_FIRES_2019_011921',
    'usfa_nfirs_2018_hazmat/USFA NFIRS 2018 Hazmat/NFIRS_FIRES_2018_110119',
    'usfa_nfirs_2017_hazmat/USFA NFIRS 2017 Hazmat/NFIRS_FIRES_2017_020719',
    'usfa_nfirs_2016_hazmat/USFA NFIRS 2016 Hazmat/NFIRS_FIRES_2016_02-05-2018',
    'usfa_nfirs_2015_hazmat/USFA NFIRS 2015 Hazmat/NFIRS_FIRES_2015_20170215',
    'usfa_nfirs_2014_hazmat/USFA NFIRS 2014 Hazmat/NFIRS_2014_030216',
    'usfa_nfirs_2013_hazmat/USFA NFIRS 2013 Hazmat/NFIRS_2013_121514',
    'usfa_nfirs_2012/USFA NFIRS 2012/NFIRS_2012_052714'
]
OUT_DIR = './data/nfirs/'

We'll create a local sqlite3 database file so that we can easily store our data as we add geocodes to the existing addresses.

In [3]:
# Create table for incidentaddresses.
conn = sqlite3.Connection(OUT_DIR + 'fire_data.db')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS incident_address')
cur.execute('DROP TABLE IF EXISTS basic_incident')
cur.execute('DROP TABLE IF EXISTS fire_incident')

cur.execute("""
    CREATE TABLE IF NOT EXISTS basic_incident (
        INCIDENT_KEY TEXT PRIMARY KEY,
        STATE TEXT,
        FDID INTEGER,
        INC_DATE TEXT,
        INC_NO INTEGER,
        EXP_NO INTEGER,
        VERSION REAL,
        DEPT_STA TEXT,
        INC_TYPE INTEGER,
        ADD_WILD INTEGER,
        AID TEXT,
        ALARM INTEGER,
        ARRIVAL TEXT,
        INC_CONT TEXT,
        LU_CLEAR REAL,
        SHIFT TEXT,
        ALARMS INTEGER,
        DISTRICT INTEGER,
        ACT_TAK1 REAL,
        ACT_TAK2 INTEGER,
        ACT_TAK3 INTEGER,
        APP_MOD TEXT,
        SUP_APP INTEGER,
        EMS_APP INTEGER,
        OTH_APP INTEGER,
        SUP_PER INTEGER,
        EMS_PER INTEGER,
        OTH_PER INTEGER,
        RESOU_AID TEXT,
        PROP_LOSS REAL,
        CONT_LOSS REAL,
        PROP_VAL REAL,
        CONT_VAL REAL,
        FF_DEATH REAL,
        OTH_DEATH REAL,
        FF_INJ REAL,
        OTH_INJ REAL,
        DET_ALERT TEXT,
        HAZ_REL TEXT,
        MIXED_USE TEXT,
        PROP_USE INTEGER,
        CENSUS INTEGER
    )""")
conn.commit()

cur.execute("""CREATE TABLE IF NOT EXISTS incident_address (
    INCIDENT_KEY TEXT PRIMARY KEY,
    STATE TEXT,
    FDID INTEGER,
    INC_DATE INTEGER,
    INC_NO INTEGER,
    EXP_NO INTEGER,
    LOC_TYPE INTEGER,
    NUM_MILE INTEGER,
    STREET_PRE TEXT,
    STREETNAME TEXT,
    STREETTYPE TEXT,
    STREETSUF TEXT,
    APT_NO TEXT,
    CITY TEXT,
    STATE_ID TEXT,
    ZIP5 INTEGER,
    ZIP4 INTEGER,
    X_STREET TEXT
)""")
conn.commit()

cur.execute("""
CREATE TABLE IF NOT EXISTS fire_incident (
    INCIDENT_KEY TEXT PRIMARY KEY,
    STATE TEXT,
    FDID INTEGER,
    INC_DATE INTEGER,
    INC_NO INTEGER,
    EXP_NO INTEGER,
    VERSION INTEGER,
    NUM_UNIT REAL,
    NOT_RES TEXT,
    BLDG_INVOL TEXT,
    ACRES_BURN REAL,
    LESS_1ACRE TEXT,
    ON_SITE_M1 TEXT,
    MAT_STOR1 TEXT,
    ON_SITE_M2 TEXT,
    MAT_STOR2 TEXT,
    ON_SITE_M3 TEXT,
    MAT_STOR3 TEXT,
    AREA_ORIG TEXT,
    HEAT_SOURC TEXT,
    FIRST_IGN TEXT,
    CONF_ORIG REAL,
    TYPE_MAT TEXT,
    CAUSE_IGN TEXT,
    FACT_IGN_1 TEXT,
    FACT_IGN_2 REAL,
    HUM_FAC_1 TEXT,
    HUM_FAC_2 REAL,
    HUM_FAC_3 REAL,
    HUM_FAC_4 REAL,
    HUM_FAC_5 REAL,
    HUM_FAC_6 REAL,
    HUM_FAC_7 REAL,
    HUM_FAC_8 REAL,
    AGE REAL,
    SEX REAL,
    EQUIP_INV TEXT,
    SUP_FAC_1 TEXT,
    SUP_FAC_2 REAL,
    SUP_FAC_3 REAL,
    MOB_INVOL TEXT,
    MOB_TYPE TEXT,
    MOB_MAKE TEXT,
    MOB_MODEL TEXT,
    MOB_YEAR REAL,
    MOB_LIC_PL TEXT,
    MOB_STATE TEXT,
    MOB_VIN_NO TEXT,
    EQ_BRAND TEXT,
    EQ_MODEL TEXT,
    EQ_SER_NO TEXT,
    EQ_YEAR REAL,
    EQ_POWER TEXT,
    EQ_PORT REAL,
    FIRE_SPRD REAL,
    STRUC_TYPE REAL,
    STRUC_STAT TEXT,
    BLDG_ABOVE REAL,
    BLDG_BELOW REAL,
    BLDG_LGTH REAL,
    BLDG_WIDTH REAL,
    TOT_SQ_FT REAL,
    FIRE_ORIG  REAL,
    ST_DAM_MIN REAL,
    ST_DAM_SIG REAL,
    ST_DAM_HVY REAL,
    ST_DAM_XTR REAL,
    FLAME_SPRD TEXT,
    ITEM_SPRD TEXT,
    MAT_SPRD TEXT,
    DETECTOR TEXT,
    DET_TYPE TEXT,
    DET_POWER TEXT,
    DET_OPERAT TEXT,
    DET_EFFECT TEXT,
    DET_FAIL TEXT,
    AES_PRES TEXT,
    AES_TYPE TEXT,
    AES_OPER TEXT,
    NO_SPR_OP REAL,
    AES_FAIL TEXT
)""")
conn.commit()

conn.close()

In [4]:
conn = sqlite3.Connection(OUT_DIR + 'fire_data.db')
cur = conn.cursor()

overall_table_size = 0

# Append each dataframe to existing table.
for i, path in enumerate(NFIRS_PATHS):
    file_name = '/basicincident.txt'
    sep = '^'       

    print(i, path, file_name)

    df = pd.read_csv(IN_DIR + path + file_name, 
                        sep=sep,
                        low_memory=False,
                        # Specify alternative text encoding.
                        encoding='ISO-8859-1')
    
    if len(df[df.duplicated(subset=['STATE','FDID','INC_DATE','INC_NO','EXP_NO'], keep=False)]) > 0:
        display(df[df.duplicated(subset=['STATE','FDID','INC_DATE','INC_NO','EXP_NO'], keep=False)])
        df = df.drop_duplicates(subset=['STATE','FDID','INC_DATE','INC_NO','EXP_NO'], keep='first')
        
    overall_table_size = overall_table_size + len(df)
    print(overall_table_size)

    # The CSVs from 2018 and earlier have 41 columns (instead of 42)
    print(len(df.columns))
    if len(df.columns) == 41:
        df['INCIDENT_KEY'] =  df.STATE.str.upper() + '_' + df.FDID.astype(str) + '_' + df.INC_DATE.astype(str) + '_' + df.INC_NO.astype(str) + '_' + df.EXP_NO.astype(str)        
    
    df.to_sql('basic_incident',
                    conn, 
                    if_exists='append', 
                    index=False)
    conn.commit()
conn.close()

0 nfirs_fire_hazmat_pdr_2021/nfirs_fire_hazmat_pdr_2021/NFIRS_FIRES_2021_111422 /basicincident.txt
2109645
42
1 nfirs_fire_hazmat_pdr_2020/nfirs_fire_hazmat_pdr_2020/NFIRS_FIRES_2020_022322 /basicincident.txt
4291515
42
2 usfa_nfirs_2019_hazmat/USFA NFIRS 2019 Hazmat/NFIRS_FIRES_2019_011921 /basicincident.txt
6513252
42
3 usfa_nfirs_2018_hazmat/USFA NFIRS 2018 Hazmat/NFIRS_FIRES_2018_110119 /basicincident.txt
8760609
41
4 usfa_nfirs_2017_hazmat/USFA NFIRS 2017 Hazmat/NFIRS_FIRES_2017_020719 /basicincident.txt
11012040
41
5 usfa_nfirs_2016_hazmat/USFA NFIRS 2016 Hazmat/NFIRS_FIRES_2016_02-05-2018 /basicincident.txt


,STATE,FDID,INC_DATE,INC_NO,EXP_NO,VERSION,DEPT_STA,INC_TYPE,ADD_WILD,AID,ALARM,ARRIVAL,INC_CONT,LU_CLEAR,SHIFT,ALARMS,DISTRICT,ACT_TAK1,ACT_TAK2,ACT_TAK3,APP_MOD,SUP_APP,EMS_APP,OTH_APP,SUP_PER,EMS_PER,OTH_PER,RESOU_AID,PROP_LOSS,CONT_LOSS,PROP_VAL,CONT_VAL,FF_DEATH,OTH_DEATH,FF_INJ,OTH_INJ,DET_ALERT,HAZ_REL,MIXED_USE,PROP_USE,CENSUS
573607,IL,CS232,8242016,0001762,0,5.0,1,412,N,N,82420160541,82420160545,NaN,8.242016e+10,R,0,BRO,45,NaN,NaN,Y,0,0,2,0,0,5,N,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,40,429,817900
573608,IL,CS232,8242016,0001762,0,5.0,1,412,N,N,82420160541,82420160545,NaN,8.242016e+10,R,0,BRO,45,NaN,NaN,Y,0,0,2,0,0,5,N,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,40,429,817900
659039,IN,02005,6132016,0016827,0,5.0,13,111,N,N,61320161649,61320161654,NaN,6.132016e+10,A,1,S13,11,73,21,N,9,0,0,15,0,0,N,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,1,NaN,NaN,449,NaN
659040,IN,02005,6132016,0016827,0,5.0,13,111,N,N,61320161649,61320161654,NaN,6.132016e+10,A,1,S13,11,73,21,N,9,0,0,15,0,0,N,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,1,NaN,NaN,449,NaN
2179877,WI,13240,4152016,0000104,0,5.0,1,111,N,3,41520162030,41520162054,NaN,4.162016e+10,C,0,Z,11,NaN,NaN,Y,1,0,0,6,0,0,N,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2179878,WI,13240,4152016,0000104,0,5.0,1,111,N,3,41520162030,41520162054,NaN,4.162016e+10,C,0,Z,11,NaN,NaN,Y,1,0,0,6,0,0,N,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


13247426
41
6 usfa_nfirs_2015_hazmat/USFA NFIRS 2015 Hazmat/NFIRS_FIRES_2015_20170215 /basicincident.txt
15407747
41
7 usfa_nfirs_2014_hazmat/USFA NFIRS 2014 Hazmat/NFIRS_2014_030216 /basicincident.txt
17524493
41
8 usfa_nfirs_2013_hazmat/USFA NFIRS 2013 Hazmat/NFIRS_2013_121514 /basicincident.txt
19528400
41
9 usfa_nfirs_2012/USFA NFIRS 2012/NFIRS_2012_052714 /basicincident.txt
21648688
41


Now we'll load our csv files, each called incidentaddress.txt, and put them in the same SQL table.

The datasets from 2013-2018 doesn't have an INCIDENT_KEY column, so we will construct one out of the other information in the dataset. This format, with five components, is consistent with the INCIDENT_KEY field in 2019-2020.

In [5]:
# 7 minutes to run
conn = sqlite3.Connection(OUT_DIR + 'fire_data.db')
cur = conn.cursor()

overall_table_size = 0

# Append each dataframe to existing table.
for i, path in enumerate(NFIRS_PATHS):
    file_name = '/incidentaddress.txt'
    sep = '^'

    df = pd.read_csv(IN_DIR + path + file_name, 
                        sep=sep,
                        low_memory=False,
                        # Specify alternative text encoding.
                        encoding='ISO-8859-1')

    if len(df[df.duplicated(subset=['STATE','FDID','INC_DATE','INC_NO','EXP_NO'], keep=False)]) > 0:
        display(df[df.duplicated(subset=['STATE','FDID','INC_DATE','INC_NO','EXP_NO'], keep=False)])
        df = df.drop_duplicates(subset=['STATE','FDID','INC_DATE','INC_NO','EXP_NO'], keep='first')
    
    overall_table_size = overall_table_size + len(df)
    print(overall_table_size)

    # The CSVs from 2018 and earlier have 17 columns (instead of 18)
    print(len(df.columns))
    if len(df.columns) == 17:
        df['INCIDENT_KEY'] =  df.STATE.str.upper() + '_' + df.FDID.astype(str) + '_' + df.INC_DATE.astype(str) + '_' + df.INC_NO.astype(str) + '_' + df.EXP_NO.astype(str)        

    df.to_sql('incident_address',
                    conn, 
                    if_exists='append', 
                    index=False)
    conn.commit()
conn.close()

2109645
18
4291515
18
6513252
18
8760609
17
11012040
17


,STATE,FDID,INC_DATE,INC_NO,EXP_NO,LOC_TYPE,NUM_MILE,STREET_PRE,STREETNAME,STREETTYPE,STREETSUF,APT_NO,CITY,STATE_ID,ZIP5,ZIP4,X_STREET
573607,IL,CS232,8242016,0001762,0,1,1817,S,14th,AVE,NaN,NaN,Broadview,IL,60155,NaN,NaN
573608,IL,CS232,8242016,0001762,0,1,1817,S,14th,AVE,NaN,NaN,Broadview,IL,60155,NaN,NaN
659039,IN,02005,6132016,0016827,0,1,111,NaN,WASHINGTON CENTER,RD,W,NaN,FORT WAYNE,IN,46825,NaN,NaN
659040,IN,02005,6132016,0016827,0,1,111,NaN,WASHINGTON CENTER,RD,W,NaN,FORT WAYNE,IN,46825,NaN,NaN
2179877,WI,13240,4152016,0000104,0,1,318,W,Main Street,NaN,NaN,NaN,Evansville,WI,53536,NaN,NaN
2179878,WI,13240,4152016,0000104,0,1,318,W,Main Street,NaN,NaN,NaN,Stoughton,WI,53589,NaN,NaN


13247426
17
15407747
17
17524493
17
19528400
17
21648688
17


In [6]:
conn = sqlite3.Connection(OUT_DIR + 'fire_data.db')
cur = conn.cursor()

overall_table_size = 0

# Append each dataframe to existing table.
for i, path in enumerate(NFIRS_PATHS):
    file_name = '/fireincident.txt'
    sep = '^'       

    print(i, path, file_name)

    df = pd.read_csv(IN_DIR + path + file_name, 
                        sep=sep,
                        low_memory=False,
                        # Specify alternative text encoding.
                        encoding='ISO-8859-1')
    
    if len(df[df.duplicated(subset=['STATE','FDID','INC_DATE','INC_NO','EXP_NO'], keep=False)]) > 0:
        display(df[df.duplicated(subset=['STATE','FDID','INC_DATE','INC_NO','EXP_NO'], keep=False)])
        df = df.drop_duplicates(subset=['STATE','FDID','INC_DATE','INC_NO','EXP_NO'], keep='first')

    overall_table_size = overall_table_size + len(df)
    print(overall_table_size)

    # The CSVs from 2018 and earlier have 80 columns (instead of 81)
    print(len(df.columns))
    if len(df.columns) == 80:
        df['INCIDENT_KEY'] =  df.STATE.str.upper() + '_' + df.FDID.astype(str) + '_' + df.INC_DATE.astype(str) + '_' + df.INC_NO.astype(str) + '_' + df.EXP_NO.astype(str)        
    
    df.to_sql('fire_incident',
                    conn, 
                    if_exists='append', 
                    index=False)
    conn.commit()
conn.close()

0 nfirs_fire_hazmat_pdr_2021/nfirs_fire_hazmat_pdr_2021/NFIRS_FIRES_2021_111422 /fireincident.txt
558385
81
1 nfirs_fire_hazmat_pdr_2020/nfirs_fire_hazmat_pdr_2020/NFIRS_FIRES_2020_022322 /fireincident.txt
1100577
81
2 usfa_nfirs_2019_hazmat/USFA NFIRS 2019 Hazmat/NFIRS_FIRES_2019_011921 /fireincident.txt
1693257
81
3 usfa_nfirs_2018_hazmat/USFA NFIRS 2018 Hazmat/NFIRS_FIRES_2018_110119 /fireincident.txt
2292430
80
4 usfa_nfirs_2017_hazmat/USFA NFIRS 2017 Hazmat/NFIRS_FIRES_2017_020719 /fireincident.txt
2896833
80
5 usfa_nfirs_2016_hazmat/USFA NFIRS 2016 Hazmat/NFIRS_FIRES_2016_02-05-2018 /fireincident.txt


,STATE,FDID,INC_DATE,INC_NO,EXP_NO,VERSION,NUM_UNIT,NOT_RES,BLDG_INVOL,ACRES_BURN,LESS_1ACRE,ON_SITE_M1,MAT_STOR1,ON_SITE_M2,MAT_STOR2,ON_SITE_M3,MAT_STOR3,AREA_ORIG,HEAT_SOURC,FIRST_IGN,CONF_ORIG,TYPE_MAT,CAUSE_IGN,FACT_IGN_1,FACT_IGN_2,HUM_FAC_1,HUM_FAC_2,HUM_FAC_3,HUM_FAC_4,HUM_FAC_5,HUM_FAC_6,HUM_FAC_7,HUM_FAC_8,AGE,SEX,EQUIP_INV,SUP_FAC_1,SUP_FAC_2,SUP_FAC_3,MOB_INVOL,MOB_TYPE,MOB_MAKE,MOB_MODEL,MOB_YEAR,MOB_LIC_PL,MOB_STATE,MOB_VIN_NO,EQ_BRAND,EQ_MODEL,EQ_SER_NO,EQ_YEAR,EQ_POWER,EQ_PORT,FIRE_SPRD,STRUC_TYPE,STRUC_STAT,BLDG_ABOVE,BLDG_BELOW,BLDG_LGTH,BLDG_WIDTH,TOT_SQ_FT,FIRE_ORIG,ST_DAM_MIN,ST_DAM_SIG,ST_DAM_HVY,ST_DAM_XTR,FLAME_SPRD,ITEM_SPRD,MAT_SPRD,DETECTOR,DET_TYPE,DET_POWER,DET_OPERAT,DET_EFFECT,DET_FAIL,AES_PRES,AES_TYPE,AES_OPER,NO_SPR_OP,AES_FAIL
204446,IN,02005,6132016,0016827,0,5.0,2.0,N,1.0,NaN,Y,315,N,NaN,NaN,NaN,NaN,76,UU,UU,NaN,NaN,U,UU,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2,5.0,0.0,NaN,NaN,40000.0,1.0,0.0,0.0,0.0,0.0,Y,NaN,NaN,1,5,4,2,1,NaN,1,NaN,NaN,0.0,NaN
204447,IN,02005,6132016,0016827,0,5.0,2.0,N,1.0,NaN,Y,315,N,NaN,NaN,NaN,NaN,76,UU,UU,NaN,NaN,U,UU,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2,5.0,0.0,NaN,NaN,40000.0,1.0,0.0,0.0,0.0,0.0,Y,NaN,NaN,1,5,4,2,1,NaN,1,NaN,NaN,0.0,NaN


3526635
80
6 usfa_nfirs_2015_hazmat/USFA NFIRS 2015 Hazmat/NFIRS_FIRES_2015_20170215 /fireincident.txt
4125996
80
7 usfa_nfirs_2014_hazmat/USFA NFIRS 2014 Hazmat/NFIRS_2014_030216 /fireincident.txt
4722517
80
8 usfa_nfirs_2013_hazmat/USFA NFIRS 2013 Hazmat/NFIRS_2013_121514 /fireincident.txt
5277188
80
9 usfa_nfirs_2012/USFA NFIRS 2012/NFIRS_2012_052714 /fireincident.txt
5877067
80


Cannonize the city column in incident_address so all the city names are upper-case.

In [7]:
conn = sqlite3.Connection(OUT_DIR + 'fire_data.db')
cur = conn.cursor()
cur.execute("""
    UPDATE incident_address
    SET CITY = UPPER(CITY)
    WHERE CITY IS NOT NULL;
""")
conn.commit()
conn.close()

Because we'll likely be joining on incident key a lot, we'll index those columns.

In [8]:
conn = sqlite3.Connection(OUT_DIR + 'fire_data.db')
cur = conn.cursor()

add_col = """
    ALTER TABLE incident_address
    ADD COLUMN address TEXT
"""

cur.execute(add_col)
conn.commit()

update = """
    UPDATE incident_address
    SET ADDRESS = 
        COALESCE(NUM_MILE, '') || ' ' ||
        COALESCE(STREET_PRE, '') || ' ' ||
        COALESCE(STREETNAME, '') || ' ' ||
        COALESCE(STREETTYPE, '') || ' ' ||
        COALESCE(STREETSUF, '')
"""

cur.execute(update)
conn.commit()

cannonize = """
    UPDATE incident_address
    SET ADDRESS = UPPER(ADDRESS)
    WHERE ADDRESS IS NOT NULL;
"""

cur.execute(cannonize)
conn.commit()

conn.close()

In [9]:
conn = sqlite3.Connection(OUT_DIR + 'fire_data.db')
cur = conn.cursor()

add_year_col = """
    ALTER TABLE incident_address
    ADD COLUMN INC_YEAR INTEGER
"""

cur.execute(add_year_col)
conn.commit()

update_year = """
    UPDATE incident_address
    SET INC_YEAR = 
        cast(substr(INC_DATE, length(INC_DATE) - 3, 4) AS integer)
"""
cur.execute(update_year)
conn.commit()

conn.close()

In [10]:
conn = sqlite3.Connection(OUT_DIR + 'fire_data.db')
cur = conn.cursor()

cur.execute('CREATE INDEX IF NOT EXISTS idx_basic_incident_incident_key ON basic_incident (INCIDENT_KEY)')
conn.commit()

cur.execute('CREATE INDEX IF NOT EXISTS idx_incident_address_incident_key ON incident_address (INCIDENT_KEY)')
conn.commit()

cur.execute('CREATE INDEX IF NOT EXISTS idx_fire_incident_incident_key ON fire_incident (INCIDENT_KEY)')
conn.commit()

cur.execute('CREATE INDEX IF NOT EXISTS idx_incident_address_address ON incident_address (ADDRESS)')
conn.commit()

cur.execute('CREATE INDEX IF NOT EXISTS idx_incident_address_apt_no ON incident_address (APT_NO)')
conn.commit()

cur.execute('CREATE INDEX IF NOT EXISTS idx_incident_address_city ON incident_address (CITY)')
conn.commit()

cur.execute('CREATE INDEX IF NOT EXISTS idx_incident_address_state ON incident_address (STATE)')
conn.commit()

cur.execute('CREATE INDEX IF NOT EXISTS idx_incident_address_zip ON incident_address (ZIP5)')
conn.commit()

cur.execute('CREATE INDEX IF NOT EXISTS idx_basic_incident_inc_type ON basic_incident (INC_TYPE)')
conn.commit()

In [12]:
conn = sqlite3.Connection(OUT_DIR + 'fire_data.db')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS buildings_fires_all')
conn.commit()

cur = conn.cursor()
query = """
    CREATE TABLE buildings_fires_all AS
    SELECT 
        bi.INCIDENT_KEY,
        ia.ADDRESS,
        ia.NUM_MILE,
        ia.STREET_PRE,
        ia.STREETNAME,
        ia.STREETTYPE,
        ia.STREETSUF,
        ia.APT_NO,
        ia.CITY,
        ia.STATE,
        ia.ZIP5,
        ia.INC_DATE,
        ia.INC_YEAR,
        ia.LOC_TYPE,
        bi.INC_TYPE, 
        bi.PROP_LOSS,
        bi.CONT_LOSS,
        bi.OTH_DEATH,
        bi.OTH_INJ,
        bi.FF_DEATH,
        bi.FF_INJ,
        bi.AID,
        bi.ALARM,
        bi.ARRIVAL,
        bi.ALARMS,
        bi.ACT_TAK1,
        bi.ACT_TAK2,
        bi.ACT_TAK3,
        bi.APP_MOD,
        bi.SUP_APP,
        bi.EMS_APP,
        bi.OTH_APP,
        bi.DET_ALERT,
        bi.HAZ_REL,
        fi.NUM_UNIT,
        fi.NOT_RES,
        fi.BLDG_INVOL,
        fi.ACRES_BURN,
        fi.LESS_1ACRE,
        fi.ON_SITE_M1,
        fi.MAT_STOR1,
        fi.ON_SITE_M2,
        fi.MAT_STOR2,
        fi.ON_SITE_M3,
        fi.MAT_STOR3,
        fi.AREA_ORIG,
        fi.HEAT_SOURC,
        fi.FIRST_IGN,
        fi.CONF_ORIG,
        fi.TYPE_MAT,
        fi.CAUSE_IGN,
        fi.FACT_IGN_1,
        fi.FACT_IGN_2,
        fi.HUM_FAC_1,
        fi.HUM_FAC_2,
        fi.HUM_FAC_3,
        fi.HUM_FAC_4,
        fi.HUM_FAC_5,
        fi.HUM_FAC_6,
        fi.HUM_FAC_7,
        fi.HUM_FAC_8,
        fi.AGE,
        fi.SEX,
        fi.EQUIP_INV,
        fi.SUP_FAC_1,
        fi.SUP_FAC_2,
        fi.SUP_FAC_3,
        fi.MOB_INVOL,
        fi.MOB_TYPE,
        fi.MOB_MAKE,
        fi.MOB_MODEL,
        fi.MOB_YEAR,
        fi.MOB_LIC_PL,
        fi.MOB_STATE,
        fi.MOB_VIN_NO,
        fi.EQ_BRAND,
        fi.EQ_MODEL,
        fi.EQ_SER_NO,
        fi.EQ_YEAR,
        fi.EQ_POWER,
        fi.EQ_PORT,
        fi.FIRE_SPRD,
        fi.STRUC_TYPE,
        fi.STRUC_STAT,
        fi.BLDG_ABOVE,
        fi.BLDG_BELOW,
        fi.BLDG_LGTH,
        fi.BLDG_WIDTH,
        fi.TOT_SQ_FT,
        fi.FIRE_ORIG,
        fi.ST_DAM_MIN,
        fi.ST_DAM_SIG,
        fi.ST_DAM_HVY,
        fi.ST_DAM_XTR,
        fi.FLAME_SPRD,
        fi.ITEM_SPRD,
        fi.MAT_SPRD,
        fi.DETECTOR,
        fi.DET_TYPE,
        fi.DET_POWER,
        fi.DET_OPERAT,
        fi.DET_EFFECT,
        fi.DET_FAIL,	
        fi.AES_PRES,
        fi.AES_TYPE,
        fi.AES_OPER,
        fi.NO_SPR_OP,	
        fi.AES_FAIL
    FROM basic_incident bi
        LEFT JOIN incident_address ia USING (INCIDENT_KEY)
        LEFT JOIN fire_incident fi USING (INCIDENT_KEY)
    WHERE bi.INC_TYPE IN (111, 113, 114, 115, 116, 118)
"""
cur.execute(query)
conn.commit()
conn.close()